In [ ]:
import json
with open('../algtest-pyprocess/measurement_stats.json', 'r') as f:
    fwinfo = json.load(f)

In [ ]:
from collections import defaultdict

records = []
years = set()

for fw, info in fwinfo.items():
    record = info.copy()
    record['vendor'], record['fw'] = fw.split()
    records.append(record)
    if "year" in record:
        years.add(record["year"])

years = {x:3 for x in sorted(years)}

# split by vendors
timelines = defaultdict(list)
for record in records:
    timelines[record['vendor']].append(record)

timelines = list(timelines.items())
timelines[0], timelines[1] = timelines[1], timelines[0]
timelines[1], timelines[2] = timelines[2], timelines[1]
timelines[2], timelines[4] = timelines[4], timelines[2]
timelines[3], timelines[5] = timelines[5], timelines[3]
timelines[3], timelines[4] = timelines[4], timelines[3]
timelines = dict(timelines)

for vendor in timelines:
    timelines[vendor].sort(key=lambda x: list(map(int, x['fw'].split('.'))))
    # interpolate years
    prev_year = 2012
    for record in timelines[vendor]:
        if 'year' not in record:
            record['year'] = prev_year
        else:
            prev_year = record['year']

# fix AMD
for record in timelines['AMD']:
    if record['fw'] in ("3.5.0.3", "3.21.0.4"):
        record["year"] = 2018
    if record['fw'] in ("2.12.0.1"):
        record["year"] = 2017

# fix IFX
for record in timelines['IFX']:
    if record['fw'] in ("5.40.7.45826"):
        record["year"] = 2015
    if record['fw'] in ("5.62.12.13824"):
        record["year"] = 2016

for vendor in timelines:
    timelines[vendor].sort(key=lambda x: [x["year"]] + list(map(int, x['fw'].split('.'))))

# fix IFX
timelines["IFX"].sort(key=lambda x: [x["year"]] + [list(map(int, x['fw'].split('.')))[1]] + [list(map(int, x['fw'].split('.')))[0]] + [list(map(int, x['fw'].split('.')))[2]] + [list(map(int, x['fw'].split('.')))[3]])

# compute max firmwares per year
for vendor in timelines:
    vendor_years = {x:0 for x in sorted(years)}

    for record in timelines[vendor]:
        vendor_years[record["year"]] += 1

    for year in vendor_years:
        years[year] = max(years[year], vendor_years[year])

# compute year positions
year_positions = {}
prev = 0
for year in sorted(years):
    year_positions[year] = prev
    prev += years[year] + 1


for vendor in timelines:
    prev_year = 2012
    offset = 0
    for record in timelines[vendor]:
        if prev_year < record["year"]:
            offset = 0
            prev_year = record["year"]

        record['position'] = year_positions[record["year"]] + offset
        offset += 1

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines

# Create a grid of subplots, with one row for each vendor
fig, axs = plt.subplots(len(timelines), 1, figsize=(12, 5), gridspec_kw={'height_ratios': [1 / len(timelines)] * len(timelines)}, constrained_layout=True, sharex=True)

for idx, (ax, vendor) in enumerate(zip(axs, timelines)):
    positions = [record['position'] for record in timelines[vendor]]

    ax.plot(positions, [0] * len(positions), "-", color="k") # base line
    ax.plot([positions[-1], year_positions[2021] + 3], [0, 0], color="k", linestyle="dotted") # continue to future
    ax.plot([year_positions[2021] + 3], [0], ">", color="k", markersize=5) # add arrow

    for record in timelines[vendor]:
        symbol = "."
        if "tpm2-algtest" in record and record["tpm2-algtest"]:
            symbol = "o"
            if "ecc" in record and record["ecc"]:
                symbol = "D"

        color = "k"
        if "tpm revision" in record and record["tpm revision"]:
            if float(record["tpm revision"]) == 1.16:
                color = "#d81b60"
            elif float(record["tpm revision"]) == 1.38:
                color = "#04ab8f"
            elif float(record["tpm revision"]) == 1.59:
                color = "#ffc107"

        ax.plot(record['position'], [0], symbol, color=color)


    for record in timelines[vendor]:
        ax.text(record['position'], 0.02, record['fw'], fontsize=8, horizontalalignment="left", verticalalignment="bottom", rotation=45)

    ax.yaxis.set_visible(False)
    if idx == len(axs) - 1:
        ax.spines[["left", "top", "right"]].set_visible(False)
        ax.xaxis.set_ticks(list(year_positions.values()), list(year_positions.keys()))
        ax.set_xlabel("Year of TPM revision conformance")
    else:
        ax.spines[["left", "top", "right", "bottom"]].set_visible(False)
        ax.xaxis.set_visible(False)

    ax.margins(y=0.2)
    ax.set_title(vendor, loc="right")

color_handles = [mpatches.Patch(facecolor=color, label=revision) for color, revision in [("#d81b60", "rev 1.16"), ("#04ab8f", "rev 1.38"), ("#ffc107", "rev 1.59"), ("k", "other")]]
line_handles = [
    mlines.Line2D([0], [0], marker='.', color='w', label='tpm_pcr', markerfacecolor='gray', markersize=10),
    mlines.Line2D([0], [0], marker='o', color='w', label='tpm2-algtest', markerfacecolor='gray', markersize=10),
    mlines.Line2D([0], [0], marker='D', color='w', label='with ECC', markerfacecolor='gray', markersize=10)
    ]
fig.legend(handles=color_handles + line_handles, loc=(0.015, 0.78), ncol=2, columnspacing=0.5, prop={'size': 9})

plt.savefig("timeline.pdf", bbox_inches='tight')
plt.show()


In [ ]:
print("Total", len(fwinfo))
print("TPM_PCR", len(list(filter(lambda x: "TPM_PCR" in x and x["TPM_PCR"], fwinfo.values()))))
print("tpm2-algtest", len(list(filter(lambda x: "tpm2-algtest" in x and x["tpm2-algtest"], fwinfo.values()))))
print("ECC", len(list(filter(lambda x: "ECC" in x and x["ECC"], fwinfo.values()))))


In [ ]:
for fw, info in sorted(fwinfo.items(), key=lambda x: x[0]):
    if "tpm2-algtest" not in info or not info["tpm2-algtest"]:
        continue
    print(fw)